## Image Generation

In [ ]:
# !pip install transformers==4.49 diffusers==0.32.2
# !wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/utils.py
# !wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/LandscapeGenerator.py

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = ""

from datasets import load_dataset
from PIL import Image as PImage

from LandscapeGenerator import LandscapeGenerator

In [ ]:
newsdata_ds = load_dataset("thiagohersan/newsdata-images", split="newsdata")
newsdata_data = [img for img in newsdata_ds]
newsdata_data_date = sorted(newsdata_data, key=lambda x: x["pubDate"], reverse=True)[:48]

In [ ]:
models = [
  "runwayml/stable-diffusion-inpainting",
  "stable-diffusion-v1-5/stable-diffusion-inpainting",
  "stabilityai/stable-diffusion-2-inpainting"
]

mLG = LandscapeGenerator(newsdata_data_date, models[1])

### New landscape

In [ ]:
mLG.gen_landscape(keep_width=256, size=(1440, 512), n=8, label="test00")

### Continue with seed

In [ ]:
seed = PImage.open("./imgs/ml20251019_184700/ml20251019_184700_07.jpg")
mLG.gen_landscape(keep_width=256, size=(1440, 512), n=12, label="test01", seed_img=seed)

### Graft

In [ ]:
limg = PImage.open("./imgs/ml20251019_00072.jpg")
rimg = PImage.open("./imgs/ml20251019_00074.jpg")
# display(limg)
# display(rimg)

keep_width=256
img_in,mask_in = mLG.prep_graft(limg, rimg, keep_width=keep_width, right_offset=32, label="ml20251019_00073")
display(img_in)
display(mask_in)

In [ ]:
output = mLG.pipe(
  prompt="flooded forest with trash",
  negative_prompt="repetitive, stretched, vertical splits, sharp transitions",
  image=img_in,
  mask_image=mask_in,
  width=img_in.size[0], height=img_in.size[1],
  guidance_scale=7.5,
  strength=1.0,
  num_inference_steps=24,
  num_images_per_prompt=4,
)

for i in output.images:
  display(i)

In [ ]:
iw,ih = output.images[2].size
output.images[2].crop((keep_width,0,iw,ih)).save("imgs/ml20251019_00073a.jpg")